In [1]:
import scipy.io as sio
import numpy as np

from __future__ import print_function

import keras.callbacks as cb
import keras
from keras.layers import Input, Activation, Dense, Dropout, Flatten, merge
from keras.layers import Convolution3D, MaxPooling3D
from keras.models import Sequential, Model
from keras.optimizers import SGD,Adadelta
from keras.regularizers import l1, l2
from keras.utils import np_utils
from keras import layers
from keras import backend as K  
from IPython.core.debugger import Tracer
from random import shuffle

%matplotlib inline
from matplotlib import pyplot as plt
import time

import pickle


Using TensorFlow backend.


In [2]:
pkl_file = open('C:\\Users\\PangTouXian\\Documents\\Python_scripts\\cnnChair\\data\\cubeTest16_1dir', 'rb')
cubeTest16 = pickle.load(pkl_file)
pkl_file.close()
pkl_file = open('C:\\Users\\PangTouXian\\Documents\\Python_scripts\\cnnChair\\data\\cubeTrain16_1dir', 'rb')
cubeTrain16 = pickle.load(pkl_file)
pkl_file.close()
cubeDim = 16

In [3]:
def PreprocessDataset():
    # Load dataset
    trainData = cubeTrain16
    testData = cubeTest16
    trainLabel =np.concatenate((np.zeros(106),np.ones(515),2*np.ones(889),3*np.ones(200),4*np.ones(200),
                                5*np.ones(465),6*np.ones(200),7*np.ones(680),8*np.ones(392),9*np.ones(344)))
    testLabel =np.concatenate((np.zeros(50),np.ones(100),2*np.ones(100),3*np.ones(86),4*np.ones(86),
                               5*np.ones(100),6*np.ones(86),7*np.ones(100),8*np.ones(100),9*np.ones(100))) 
    print (trainData.shape)
    print(trainLabel.shape) 
    
    # Transform lables to one-hot
    print(trainLabel[0])
    trainLabel = np_utils.to_categorical(trainLabel, 10)
    testLabel = np_utils.to_categorical(testLabel, 10)
    print(trainLabel[0])
    print(trainLabel.shape)
    print(testLabel.shape)
    
    # Reshape
    trainData = trainData.reshape(trainData.shape[0], cubeDim,cubeDim,cubeDim, 1)
    testData = testData.reshape(testData.shape[0], cubeDim,cubeDim,cubeDim, 1)
    print(trainData.shape)
    return [trainData,testData,trainLabel,testLabel]

trainData,testData,trainLabel,testLabel = PreprocessDataset()

(3991, 16, 16, 16)
(3991,)
0.0
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(3991, 10)
(908, 10)
(3991, 16, 16, 16, 1)


In [8]:
def DefineModel_1():
    input = Input(shape=(cubeDim,cubeDim,cubeDim,1))
    
    conv1a = Convolution3D(64,3,3,3,strides=(1, 1, 1), activation='relu', 
                          padding='same')(input)
    conv1b = Convolution3D(64,2,2,2,strides=(1, 1, 1), activation='relu', 
                          padding='same')(input)
    merged1 = merge([conv1a,conv1b],mode='concat',concat_axis=-1)
    print(np.shape(merged1))
    pooling1 = MaxPooling3D(pool_size=(2, 2, 2))(merged1)
    dropout = Dropout(0.5)(pooling1)
    
    conv2a = Convolution3D(64,1,1,1,strides=(1, 1, 1), activation='relu', 
                          padding='same')(pooling1)
    conv2b = Convolution3D(128,3,3,3,strides=(1, 1, 1), activation='relu', 
                          padding='same')(conv2a)
    conv2c = Convolution3D(64,1,1,1,strides=(1, 1, 1), activation='relu', 
                          padding='same')(pooling1)
    conv2d = Convolution3D(128,2,2,2,strides=(1, 1, 1), activation='relu', 
                          padding='same')(conv2c)
    conv2e = Convolution3D(64,1,1,1,strides=(1, 1, 1), activation='relu', 
                          padding='same')(pooling1) 
    merged2 = merge([conv2b,conv2d,conv2e],mode='concat',concat_axis=-1)
    print(np.shape(merged2))
    pooling2 = MaxPooling3D(pool_size=(2, 2, 2))(merged2)
    dropout = Dropout(0.5)(pooling2)
    
    conv3a = Convolution3D(128,1,1,1,strides=(1, 1, 1), activation='relu', 
                          padding='same')(pooling2)
    conv3b = Convolution3D(256,3,3,3,strides=(1, 1, 1), activation='relu', 
                          padding='same')(conv3a)
    conv3c = Convolution3D(128,1,1,1,strides=(1, 1, 1), activation='relu', 
                          padding='same')(pooling2)
    conv3d = Convolution3D(256,2,2,2,strides=(1, 1, 1), activation='relu', 
                          padding='same')(conv3c)
    conv3e = Convolution3D(128,1,1,1,strides=(1, 1, 1), activation='relu', 
                          padding='same')(pooling2)
    merged3 = merge([conv3b,conv3d,conv3e],mode='concat',concat_axis=-1)
    print(np.shape(merged3))
    flatten2 = Flatten()(merged3)    
    
    dense1 = Dense(256,activation='relu')(flatten2)
    dropout = Dropout(0.5)(dense1)
    dense2 = Dense(10, activation='softmax')(dropout)
    model = Model(inputs=input, outputs=dense2)
    
    optmzr = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=optmzr, metrics=["accuracy"])

 
    return model

model = DefineModel_1()
print(model.output_shape)


C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), activation="relu", padding="same", strides=(1, 1, 1))`
  """
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (2, 2, 2), activation="relu", padding="same", strides=(1, 1, 1))`
  import sys
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:8: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\keras\legacy\layers.py:464: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc

(?, 16, 16, 16, 128)
(?, 8, 8, 8, 320)


C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:31: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(256, (3, 3, 3), activation="relu", padding="same", strides=(1, 1, 1))`
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (1, 1, 1), activation="relu", padding="same", strides=(1, 1, 1))`
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(256, (2, 2, 2), activation="relu", padding="same", strides=(1, 1, 1))`
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (1, 1, 1), activation="relu", padding="same", strides=(1, 1, 1))`


(?, 4, 4, 4, 640)
(None, 10)


C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:38: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [13]:
def TrainModel_1(data=None, epochs=15, batch=128):
    start_time = time.time()
    model = DefineModel_1()
    if data is None:
        print("Must provide data.")
        return
    trainData,testData,trainLabel,testLabel = data
    print('Start training.')

    history = model.fit(trainData, trainLabel, nb_epoch=epochs, batch_size=batch,
              validation_data=(testData, testLabel),verbose=1,shuffle=True)
    print("Training took {0} seconds.".format(time.time() - start_time))
    return model, history

In [14]:
trained_model_1, training_history_1 = TrainModel_1(data=[trainData,testData,trainLabel,testLabel])

C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), activation="relu", padding="same", strides=(1, 1, 1))`
  """
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (2, 2, 2), activation="relu", padding="same", strides=(1, 1, 1))`
  import sys
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:8: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\keras\legacy\layers.py:464: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc

(?, 16, 16, 16, 128)
(?, 8, 8, 8, 320)


C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:31: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(256, (3, 3, 3), activation="relu", padding="same", strides=(1, 1, 1))`
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (1, 1, 1), activation="relu", padding="same", strides=(1, 1, 1))`
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(256, (2, 2, 2), activation="relu", padding="same", strides=(1, 1, 1))`
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (1, 1, 1), activation="relu", padding="same", strides=(1, 1, 1))`
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\sit

(?, 4, 4, 4, 640)
Start training.
Train on 3991 samples, validate on 908 samples
Epoch 1/15
3991/3991 [==============================] - 12s 3ms/step - loss: 0.8816 - acc: 0.7141 - val_loss: 0.5708 - val_acc: 0.7621
Epoch 2/15
3991/3991 [==============================] - 11s 3ms/step - loss: 0.3089 - acc: 0.8970 - val_loss: 0.3764 - val_acc: 0.8557
Epoch 3/15
3991/3991 [==============================] - 11s 3ms/step - loss: 0.2029 - acc: 0.9313 - val_loss: 0.3512 - val_acc: 0.8700
Epoch 4/15
3991/3991 [==============================] - 11s 3ms/step - loss: 0.1685 - acc: 0.9421 - val_loss: 0.3823 - val_acc: 0.8744
Epoch 5/15
3991/3991 [==============================] - 11s 3ms/step - loss: 0.1231 - acc: 0.9551 - val_loss: 0.3419 - val_acc: 0.8744
Epoch 6/15
3991/3991 [==============================] - 11s 3ms/step - loss: 0.0973 - acc: 0.9649 - val_loss: 0.3313 - val_acc: 0.8954
Epoch 7/15
3991/3991 [==============================] - 11s 3ms/step - loss: 0.0798 - acc: 0.9709 - val_loss:

In [20]:
#new test

num_filters1 = 64
num_filters2 = 128
filter_size1 = 3
filter_size2 = 2

def DefineModel_2():
    input = Input(shape=(cubeDim,cubeDim,cubeDim,1))
    conv1a = Convolution3D(num_filters1, filter_size1,filter_size1,filter_size1, strides=(1, 1, 1), activation='relu', 
                          padding='valid',input_shape=(cubeDim,cubeDim,cubeDim, 1))(input)
    conv1b = Convolution3D(num_filters1, filter_size1,filter_size1,filter_size1, activation='relu')(conv1a)
    pooling1a = MaxPooling3D(pool_size=(2, 2, 2))(conv1b)
    print(np.shape(conv1a))    
    
    conv1c = Convolution3D(num_filters2, filter_size1,filter_size1,filter_size1, strides=(1, 1, 1), activation='relu', 
                          padding='valid')(pooling1a)
    pooling1b = MaxPooling3D(pool_size=(2, 2, 2))(conv1c)
    '''conv1d = Convolution3D(num_filters2, filter_size1,filter_size1,filter_size1, strides=(1, 1, 1), activation='relu', 
                          padding='valid')(conv1c)'''
    '''pooling1b = MaxPooling3D(pool_size=(2, 2, 2))(conv1d)'''

    
    ''''''
    flatten1 = Flatten()(pooling1b)
    
    conv2a = Convolution3D(num_filters1, filter_size2, filter_size2, filter_size2,strides=(1, 1, 1), activation='relu', 
                          padding='valid',input_shape=(cubeDim,cubeDim,cubeDim, 1))(input)
    conv2b = Convolution3D(num_filters1, filter_size2, filter_size2, filter_size2,strides=(1, 1, 1), activation='relu', 
                          padding='valid')(conv2a)
    pooling2a = MaxPooling3D(pool_size=(2, 2, 2))(conv2b)
    
    conv2c = Convolution3D(num_filters2, filter_size2, filter_size2, filter_size2,strides=(1, 1, 1), activation='relu', 
                          padding='valid')(pooling2a)
    conv2d = Convolution3D(num_filters2, filter_size2, filter_size2, filter_size2,strides=(1, 1, 1), activation='relu', 
                          padding='valid')(conv2c)
    conv2e = Convolution3D(num_filters2, filter_size2, filter_size2, filter_size2,strides=(1, 1, 1), activation='relu', 
                          padding='valid')(conv2d)
    pooling2b = MaxPooling3D(pool_size=(2, 2, 2))(conv2e)
    print(np.shape(pooling2b))
    dropout2 = Dropout(0.5)(pooling2b)
    
    
    ''''''
    flatten2 = Flatten()(dropout2)
    merged = merge([flatten1, flatten2],mode='concat',concat_axis=1,name='merged')
    
    '''merged = keras.layers.concatenate([dropout1, dropout2],axis=-1)'''

    '''flatten = Flatten()(merged)'''
    
    dense1 = Dense(128,activation='relu')(merged)
    dropout = Dropout(0.5)(dense1)
    dense2 = Dense(10, activation='softmax')(dropout)
    
    model = Model(inputs=input, outputs=dense2)
    
    ''' optmzr = SGD(lr=0.1, clipnorm=5.)''' '''--88%'''
    
    optmzr = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=optmzr, metrics=["accuracy"])
    '''
    
    optmzr = keras.optimizers.Adagrad(lr=0.01, epsilon=1e-08, decay=0.0)   
    model.compile(loss='categorical_crossentropy', optimizer=optmzr, metrics=["accuracy"])'''
    return model


model = DefineModel_2()
print(model.output_shape)


C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), strides=(1, 1, 1), activation="relu", padding="valid", input_shape=(16, 16, 1...)`
  # This is added back by InteractiveShellApp.init_path()
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), activation="relu")`
  if sys.path[0] == '':
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (3, 3, 3), strides=(1, 1, 1), activation="relu", padding="valid")`
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (2, 2, 2), strides=(1, 1, 1), activation="relu", padd

(?, 14, 14, 14, 64)
(?, 2, 2, 2, 128)
(None, 10)


C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\keras\legacy\layers.py:464: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [21]:
def TrainModel_2(data=None, epochs=20, batch=128):
    start_time = time.time()
    model = DefineModel_2()
    if data is None:
        print("Must provide data.")
        return
    trainData,testData,trainLabel,testLabel = data
    print('Start training.')

    history = model.fit(trainData, trainLabel, nb_epoch=epochs, batch_size=batch,
              validation_data=(testData, testLabel),verbose=1,shuffle=True)
    print("Training took {0} seconds.".format(time.time() - start_time))
    return model, history

In [22]:
trained_model_2, training_history_2 = TrainModel_2(data=[trainData,testData,trainLabel,testLabel])

C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), strides=(1, 1, 1), activation="relu", padding="valid", input_shape=(16, 16, 1...)`
  # This is added back by InteractiveShellApp.init_path()
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), activation="relu")`
  if sys.path[0] == '':
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (3, 3, 3), strides=(1, 1, 1), activation="relu", padding="valid")`
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (2, 2, 2), strides=(1, 1, 1), activation="relu", padd

(?, 14, 14, 14, 64)
(?, 2, 2, 2, 128)


C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:46: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\keras\legacy\layers.py:464: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:11: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # This is added back by InteractiveShellApp.init_path()


Start training.
Train on 3991 samples, validate on 908 samples
Epoch 1/20
3991/3991 [==============================] - 12s 3ms/step - loss: 1.2516 - acc: 0.5866 - val_loss: 0.6201 - val_acc: 0.7952
Epoch 2/20
3991/3991 [==============================] - 10s 3ms/step - loss: 0.5328 - acc: 0.8371 - val_loss: 0.4956 - val_acc: 0.8458
Epoch 3/20
3991/3991 [==============================] - 10s 3ms/step - loss: 0.3880 - acc: 0.8725 - val_loss: 0.4603 - val_acc: 0.8579
Epoch 4/20
3991/3991 [==============================] - 10s 3ms/step - loss: 0.3027 - acc: 0.9038 - val_loss: 0.4052 - val_acc: 0.8513
Epoch 5/20
3991/3991 [==============================] - 11s 3ms/step - loss: 0.2549 - acc: 0.9085 - val_loss: 0.3641 - val_acc: 0.8700
Epoch 6/20
3991/3991 [==============================] - 11s 3ms/step - loss: 0.2378 - acc: 0.9221 - val_loss: 0.3069 - val_acc: 0.8932
Epoch 7/20
3991/3991 [==============================] - 11s 3ms/step - loss: 0.1813 - acc: 0.9389 - val_loss: 0.3262 - val_acc:

In [10]:
def TestModel(model=None, data=None):
    if model is None:
        print("Must provide a trained model.")
        return
    if data is None:
        print("Must provide data.")
        return
    
    result =model.predict_on_batch(np.reshape(data,[1,16,16,16,1]))
    return result

In [14]:
index = [50,150,250,336,422,522,608,708,808,908]
n = 0
i = index[n]
cl = 1
res = 0
for num in range (0,908):
    result1 = TestModel(trained_model_1, testData[num,:])
    result2 = TestModel(trained_model_2, testData[num,:])
    r_cl = np.argmax(result1+result2)+1
    if num>=i:
        n = n+1
        i = index[n]
        cl = cl+1
    if r_cl==cl:
        res = res +1
        
print(res/908)

0.8975770925110133


In [5]:
#new test

num_filters1 = 64
num_filters2 = 128
filter_size1 = 3
filter_size2 = 2

def DefineModel_3():
    input = Input(shape=(cubeDim,cubeDim,cubeDim,1))
    conv1a = Convolution3D(num_filters1, filter_size1,filter_size1,filter_size1, strides=(1, 1, 1), activation='relu', 
                          padding='same',input_shape=(cubeDim,cubeDim,cubeDim, 1))(input)
    conv1b = Convolution3D(num_filters1, filter_size1,filter_size1,filter_size1, padding='same',activation='relu')(conv1a)
    pooling1a = MaxPooling3D(pool_size=(2, 2, 2))(conv1b)

    conv1c = Convolution3D(num_filters2, filter_size1,filter_size1,filter_size1, strides=(1, 1, 1), activation='relu', 
                          padding='same')(pooling1a)
    conv1d = Convolution3D(num_filters2, filter_size1,filter_size1,filter_size1, strides=(1, 1, 1), activation='relu', 
                          padding='same')(conv1c)
    pooling1b = MaxPooling3D(pool_size=(2, 2, 2))(conv1d)

    dropout1 = Dropout(0.5)(pooling1b)
    
    ''''''
    flatten1 = Flatten()(dropout1)
    
    conv2a = Convolution3D(num_filters1, filter_size2, filter_size2, filter_size2,strides=(1, 1, 1), activation='relu', 
                          padding='same',input_shape=(cubeDim,cubeDim,cubeDim, 1))(input)
    conv2b = Convolution3D(num_filters1, filter_size2, filter_size2, filter_size2,strides=(1, 1, 1), activation='relu', 
                          padding='same')(conv2a)
    pooling2a = MaxPooling3D(pool_size=(2, 2, 2))(conv2b)
    
    conv2c = Convolution3D(num_filters2, filter_size2, filter_size2, filter_size2,strides=(1, 1, 1), activation='relu', 
                          padding='same')(pooling2a)
    conv2d = Convolution3D(num_filters2, filter_size2, filter_size2, filter_size2,strides=(1, 1, 1), activation='relu', 
                          padding='same')(conv2c)

    pooling2b = MaxPooling3D(pool_size=(2, 2, 2))(conv2d)
    dropout2 = Dropout(0.5)(pooling2b)
    
    
    ''''''
    flatten2 = Flatten()(dropout2)
    merged = merge([flatten1, flatten2],mode='concat',concat_axis=1,name='merged')
    
    '''merged = keras.layers.concatenate([dropout1, dropout2],axis=-1)'''

    '''flatten = Flatten()(merged)'''
    
    dense1 = Dense(128,activation='relu')(merged)
    dropout = Dropout(0.5)(dense1)
    dense2 = Dense(10, activation='softmax')(dropout)
    
    model = Model(inputs=input, outputs=dense2)
    
    ''' optmzr = SGD(lr=0.1, clipnorm=5.)''' '''--88%'''
    
    optmzr = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=optmzr, metrics=["accuracy"])
    '''
    
    optmzr = keras.optimizers.Adagrad(lr=0.01, epsilon=1e-08, decay=0.0)   
    model.compile(loss='categorical_crossentropy', optimizer=optmzr, metrics=["accuracy"])'''
    return model


model = DefineModel_3()
print(model.output_shape)


C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), activation="relu", padding="same", input_shape=(16, 16, 1..., strides=(1, 1, 1))`
  # This is added back by InteractiveShellApp.init_path()
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), activation="relu", padding="same")`
  if sys.path[0] == '':
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (3, 3, 3), activation="relu", padding="same", strides=(1, 1, 1))`
  app.launch_new_instance()
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (3, 3, 3),

(None, 10)


C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (2, 2, 2), activation="relu", padding="same", strides=(1, 1, 1))`
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (2, 2, 2), activation="relu", padding="same", strides=(1, 1, 1))`
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:43: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\keras\legacy\layers.py:464: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [6]:
def TrainModel_3(data=None, epochs=20, batch=128):
    start_time = time.time()
    model = DefineModel_3()
    if data is None:
        print("Must provide data.")
        return
    trainData,testData,trainLabel,testLabel = data
    print('Start training.')

    history = model.fit(trainData, trainLabel, nb_epoch=epochs, batch_size=batch,
              validation_data=(testData, testLabel),verbose=1,shuffle=True)
    print("Training took {0} seconds.".format(time.time() - start_time))
    return model, history

In [7]:
trained_model_3, training_history_3 = TrainModel_3(data=[trainData,testData,trainLabel,testLabel])

C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), activation="relu", padding="same", input_shape=(16, 16, 1..., strides=(1, 1, 1))`
  # This is added back by InteractiveShellApp.init_path()
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), activation="relu", padding="same")`
  if sys.path[0] == '':
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (3, 3, 3), activation="relu", padding="same", strides=(1, 1, 1))`
  app.launch_new_instance()
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (3, 3, 3),

Start training.
Train on 3991 samples, validate on 908 samples
Epoch 1/20
3991/3991 [==============================] - 36s 9ms/step - loss: 1.1115 - acc: 0.6477 - val_loss: 0.5956 - val_acc: 0.7885
Epoch 2/20
3991/3991 [==============================] - 26s 6ms/step - loss: 0.4211 - acc: 0.8634 - val_loss: 0.4420 - val_acc: 0.8436
Epoch 3/20
3991/3991 [==============================] - 26s 7ms/step - loss: 0.3224 - acc: 0.8905 - val_loss: 0.3837 - val_acc: 0.8744
Epoch 4/20
3991/3991 [==============================] - 26s 7ms/step - loss: 0.2377 - acc: 0.9173 - val_loss: 0.3473 - val_acc: 0.8844
Epoch 5/20
3991/3991 [==============================] - 26s 7ms/step - loss: 0.2044 - acc: 0.9291 - val_loss: 0.3555 - val_acc: 0.8811
Epoch 6/20
3991/3991 [==============================] - 27s 7ms/step - loss: 0.1685 - acc: 0.9406 - val_loss: 0.3883 - val_acc: 0.8822
Epoch 7/20
3991/3991 [==============================] - 27s 7ms/step - loss: 0.1680 - acc: 0.9461 - val_loss: 0.3685 - val_acc:

In [11]:
def TestModel(model=None, data=None):
    if model is None:
        print("Must provide a trained model.")
        return
    if data is None:
        print("Must provide data.")
        return
    
    result =model.predict_on_batch(np.reshape(data,[1,16,16,16,1]))
    return result

In [ ]:
index = [50,150,250,336,422,522,608,708,808,908]
n = 0
i = index[n]
cl = 1
res = 0
for num in range (0,908):
    result1 = TestModel(trained_model_1, testData[num,:])
    result2 = TestModel(trained_model_3, testData[num,:])
    r_cl = np.argmax(result1+result2)+1
    if num>=i:
        n = n+1
        i = index[n]
        cl = cl+1
    if r_cl==cl:
        res = res +1
        
print(res/908)